In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import expipe
from expipe import Browser
import pandas as pd
import numpy as np
import sys
import exdir

sys.path.append('../ca2-mec') if '../ca2-mec' not in sys.path else None 
import dataloader as dl
from utils import *

In [3]:
project = expipe.get_project(dl.project_path())
include_actions = ['011-120321-2', '011-120321-3', '011-120321-4', '011-120321-5', '011-120321-6'] # choose actions to include

### Create pandas dataframe layout

In [9]:
tot_time = ['unspecified']*len(include_actions)
top_right_time = ['unspecified']*len(include_actions)
bottom_left_time = ['unspecified']*len(include_actions)
sdi = ['unspecified']*len(include_actions)

data = {'action_id': include_actions, 'total-time-in-seconds': tot_time, \
        'time-spent-in-top-social-zone': top_right_time, 'time-spent-in-bottom-social-zone': bottom_left_time, \
        'social-discrimination-index': sdi}
df = pd.DataFrame(data)

### Populate pandas dataframe with social statistics

In [10]:
def process(lim=[0,1200],margin=0.4):
    def process_row(row):
        x,y,t,_ = dl.load_tracking(row['action_id'], lim=lim, ca2_transform_data=True).T
        spikes = dl.load_spiketrains(row['action_id'], load_spikes=False, lim=lim, identify_neurons=True)

        social_types = social_label(spikes[0])
        box_locations = ~(social_types== -1) # True where there is 'box', false on 'nobox'
        box_idx_masks = np.append(box_locations, False)
        cms = corner_masks(x=x, y=y, margin=margin)
        sm1,sm2 = cms[:,box_idx_masks].T

        total_recording_time = t[-1] - t[0]
        samp_freq = len(t) / total_recording_time
        top_right_time = np.sum(sm1) / samp_freq
        bottom_left_time = np.sum(sm2) / samp_freq
        sdi = (top_right_time - bottom_left_time) / (top_right_time + bottom_left_time)

        # Save values
        row['total-time-in-seconds'] = total_recording_time
        row['time-spent-in-top-social-zone'] = top_right_time
        row['time-spent-in-bottom-social-zone'] = bottom_left_time
        row['social-discrimination-index'] = sdi
        print(row)
        
    return process_row

### Save as expipe action csv

In [11]:
df.apply(process(lim=[0,1200],margin=0.4), axis=1)
dataframes = project.require_action('dataframes')
dataframes.data['sessions'] = 'sessions.csv'
df.to_csv(dataframes.data_path('sessions'), index=False)

action_id                           011-120321-2
total-time-in-seconds                    1199.99
time-spent-in-top-social-zone            143.039
time-spent-in-bottom-social-zone         187.448
social-discrimination-index            -0.134376
Name: 0, dtype: object
action_id                           011-120321-3
total-time-in-seconds                    1199.97
time-spent-in-top-social-zone            461.606
time-spent-in-bottom-social-zone         104.749
social-discrimination-index             0.630094
Name: 1, dtype: object
action_id                           011-120321-4
total-time-in-seconds                    1199.98
time-spent-in-top-social-zone            95.0692
time-spent-in-bottom-social-zone         343.007
social-discrimination-index             -0.56597
Name: 2, dtype: object
action_id                           011-120321-5
total-time-in-seconds                    1199.99
time-spent-in-top-social-zone            123.409
time-spent-in-bottom-social-zone         196.998
